# Aug 15 Notes

## Stage 04 - Data Acquisition and Ingestion
Goals: API pull, scraping, secrets via .env, validation, saving to data/raw/

In [1]:
# API - Application Programmin Interface 
# a connection to grab and recieve data
# sometimes the data, let' say an article posted changed the format of data - its not reusable and they dont realize others are using it 
# TOS - terms of service
# Robots.txt - expectation, rate limit, not enforced 
# API is maintained, web page not maintained 
# more structured 
# timout
# In header you send your details
# 

In [4]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests 
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

DATA_RAW = pathlib.Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

load_dotenv()

ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))


Loaded ALPHAVANTAGE_API_KEY? True


In [13]:
# Timestamps 
def safe_stamp():
    return dt.datatime.now().strftime("%Y%m%d-%H%M%S") # give it to me in this format 
safe_stamp

<function __main__.safe_stamp()>

In [14]:
# tells us the current time
dt.datetime.now.strftime("%y-%m-%d")

AttributeError: 'builtin_function_or_method' object has no attribute 'strftime'

In [ ]:
 # NOA - weather API
# List comprehensions use optimised C ++ in background thus faster than regular for loops 
# entire ternary should come together if using else, only if can come at the back 
a = [c if c == 3 else 4 for c in dataset]
b = [c for c in datatset if c == 3] # after this if we put else then error 


In [ ]:
# csv - raw data 
# parquet - processed

fillna()
Purpose: fillna() specifically targets and replaces missing values (e.g., NaN, None). 
It's designed for imputation. You provide a value, method, or dictionary, and it fills in all the missing data points.

Syntax Example:

Python
df.fillna(value=0) # Fills all NaN with 0
df.fillna(method='ffill') # Fills NaN with the previous valid value
mask()
Purpose: mask() is a more general and flexible method. It replaces values based on a boolean condition you provide, whether the value is missing or not. It's the inverse of where(). You tell it what to "hide" or replace, and it does so.

Syntax Example:

Python
df.mask(df > 50, 0) # Replaces all values greater than 50 with 0
df.mask(df.isnull(), 0) # This is a roundabout way to do what fillna() does
Key Differences
Target: fillna() only works on NaN or missing values. mask() works on any value that meets a specified condition.

Condition: fillna() implicitly uses the condition of "is a missing value." mask() requires an explicit boolean condition (e.g., df > 50, df['column'] == 'bad data').

Use Case: Use fillna() for imputation and general missing data cleaning. Use mask() for conditional replacement of any data that doesn't fit your criteria, including non-missing values.

Both fillna() and mask() return a new DataFrame by default, leaving the original DataFrame unchanged.

To modify the DataFrame in place (i.e., to make the changes persist on the original object), you must explicitly use the inplace=True argument.

Perfect 👍 let’s do it with **Pandas syntax** so it’s crystal clear.

First, imagine your DataFrame has missing values:

```python
import pandas as pd

data = {'toys': ['🐻', None, None, '🐱']}
df = pd.DataFrame(data)
print(df)
```

Output:

```
   toys
0    🐻
1  None
2  None
3    🐱
```

---

### 1. **Forward Fill (copy from the left)**

```python
df['toys'].fillna(method='ffill', inplace=True)
```

Result:

```
0    🐻
1    🐻
2    🐻
3    🐱
```

---

### 2. **Back Fill (copy from the right)**

```python
df['toys'].fillna(method='bfill', inplace=True)
```

Result:

```
0    🐻
1    🐱
2    🐱
3    🐱
```

---

### 3. **Fill (with a fixed value)**

```python
df['toys'].fillna('🍎', inplace=True)
```

Result:

```
0    🐻
1    🍎
2    🍎
3    🐱
```

---

👉 So the syntax is:

* `fillna(method='ffill')` → forward fill
* `fillna(method='bfill')` → back fill
* `fillna(value)` → fill with a constant

Do you want me to also show you how this works on a **2D DataFrame (multiple columns)** with both forward and back fill?


Got it 👍 let’s make this super simple — like you’re 5.

Imagine you have a row of **empty boxes** (your DataFrame), but some boxes are missing toys (that’s your missing data, `NaN`).

### 1. **Forward Fill (ffill)** 🏃‍♀️➡️

Take the toy from the **previous box** and **copy it forward** into the empty one.

Example:

```
Boxes:   [🐻,  ,  , 🐱]
ffill → [🐻, 🐻, 🐻, 🐱]
```

### 2. **Back Fill (bfill)** ⬅️🏃‍♀️

Take the toy from the **next box** and **copy it backward** into the empty one.

Example:

```
Boxes:   [🐻,  ,  , 🐱]
bfill → [🐻, 🐱, 🐱, 🐱]
```

### 3. **Fill (with a value)** 🎁

Instead of borrowing from neighbors, you just **give every empty box the same toy** (a chosen value).

Example (fill with 🍎):

```
Boxes:   [🐻,  ,  , 🐱]
fill →  [🐻, 🍎, 🍎, 🐱]
```

---

👉 So:

* **Forward fill** = copy from the left.
* **Back fill** = copy from the right.
* **Fill** = put in a toy/value you decide.

Do you want me to also show you how this looks in **Pandas code** with a toy-like example?


ALL CREATE A NEW DATA FRAME, ORIGINAL NOT MODIFIED 

Direct access to change in the OG dataframe 
df.loc[2:3, 'price'] = 7
in that column price from indexes 2 to 3 change it to 7 
loc is direct access by row name and column name 
df['price'] = series 
0 234
1 234
not a df 
iloc - for index based 
for columns: 
df.columns 
for rows:
df.index